In [29]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [30]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [31]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/Glasses & Sunglasses - Train.npz')
data_validation = np.load(r'Full Dataset/Glasses & Sunglasses - Validation.npz')
data_test = np.load(r'Full Dataset/Glasses & Sunglasses - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_validation['images']
labels_val = data_validation['labels']


images_test = data_test['images']
labels_test = data_test['labels']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [32]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 15

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete([32, 64, 96, 128]))

METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/Model_1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [33]:
def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(32, 3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(2)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    log_dir = f"Logs/Model_1/fit/run-{session_num}"

    
        
        
        
#Plot_to_image

    def plot_confusion_matrix(cm, class_names):
 
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Compute the labels from the normalized confusion matrix.
        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    



    def plot_to_image(figure):
       
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image


        
    # log the confusion matrix as a heatmap
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')


    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        val_pred_raw = model.predict(images_val)
        val_pred = np.argmax(val_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = confusion_matrix(labels_val, val_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["Glasses", "Sunglassses"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion_matrix", cm_image, step=epoch)

    # Define the per-epoch callback.
    cm_callback = LambdaCallback(on_epoch_end=log_confusion_matrix)


    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

    # Save the model
    model.save(f"saved_model/Model_1/fit/run-{session_num}.h5")


    return accuracy



In [34]:
#Log the hyperparameter to the TensorBoard



def run(log_dir, hparams,session_num):
    hyperparameter_writer = tf.summary.create_file_writer(log_dir)
    
    
    with hyperparameter_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
        
        
        
        
#Train the model with the different hyperparameters
session_num = 1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1
       
        

       
        
        
        








---Starting trial: run-1
{'filter_size': 3, 'filter_num': 32}
Epoch 1/15
16/16 [==============================] - 1s 49ms/step
63/63 - 24s - loss: 0.3126 - accuracy: 0.8818 - val_loss: 0.2050 - val_accuracy: 0.9460 - 24s/epoch - 381ms/step
Epoch 2/15
16/16 [==============================] - 1s 48ms/step
63/63 - 22s - loss: 0.1645 - accuracy: 0.9470 - val_loss: 0.2203 - val_accuracy: 0.9620 - 22s/epoch - 344ms/step
Epoch 3/15
16/16 [==============================] - 0s 26ms/step
63/63 - 18s - loss: 0.1381 - accuracy: 0.9593 - val_loss: 0.2229 - val_accuracy: 0.9580 - 18s/epoch - 284ms/step
16/16 [==============================] - 0s 28ms/step - loss: 0.2050 - accuracy: 0.9460
---Starting trial: run-2
{'filter_size': 3, 'filter_num': 64}


C:\Users\chiez\anaconda3\envs\fashion_labeling\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/15
16/16 [==============================] - 1s 28ms/step
63/63 - 13s - loss: 0.3708 - accuracy: 0.8523 - val_loss: 0.2097 - val_accuracy: 0.9500 - 13s/epoch - 206ms/step
Epoch 2/15
16/16 [==============================] - 0s 29ms/step
63/63 - 12s - loss: 0.1572 - accuracy: 0.9490 - val_loss: 0.2125 - val_accuracy: 0.9400 - 12s/epoch - 186ms/step
Epoch 3/15
16/16 [==============================] - 0s 30ms/step
63/63 - 12s - loss: 0.1548 - accuracy: 0.9490 - val_loss: 0.2100 - val_accuracy: 0.9420 - 12s/epoch - 187ms/step
16/16 [==============================] - 0s 26ms/step - loss: 0.2097 - accuracy: 0.9500
---Starting trial: run-3
{'filter_size': 3, 'filter_num': 96}


C:\Users\chiez\anaconda3\envs\fashion_labeling\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/15
16/16 [==============================] - 1s 28ms/step
63/63 - 13s - loss: 0.4012 - accuracy: 0.8226 - val_loss: 0.2009 - val_accuracy: 0.9540 - 13s/epoch - 203ms/step
Epoch 2/15
16/16 [==============================] - 0s 29ms/step
63/63 - 12s - loss: 0.1664 - accuracy: 0.9435 - val_loss: 0.2017 - val_accuracy: 0.9580 - 12s/epoch - 194ms/step
Epoch 3/15
16/16 [==============================] - 1s 30ms/step
63/63 - 12s - loss: 0.1414 - accuracy: 0.9543 - val_loss: 0.2092 - val_accuracy: 0.9380 - 12s/epoch - 190ms/step
16/16 [==============================] - 1s 28ms/step - loss: 0.2009 - accuracy: 0.9540
---Starting trial: run-4
{'filter_size': 3, 'filter_num': 128}


...

C:\Users\chiez\anaconda3\envs\fashion_labeling\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/15
16/16 [==============================] - 1s 31ms/step
63/63 - 17s - loss: 0.3437 - accuracy: 0.8621 - val_loss: 0.4847 - val_accuracy: 0.8560 - 17s/epoch - 270ms/step
Epoch 2/15
16/16 [==============================] - 1s 32ms/step
63/63 - 15s - loss: 0.1811 - accuracy: 0.9373 - val_loss: 0.2149 - val_accuracy: 0.9380 - 15s/epoch - 232ms/step
Epoch 3/15
16/16 [==============================] - 1s 34ms/step
63/63 - 15s - loss: 0.1484 - accuracy: 0.9535 - val_loss: 0.2019 - val_accuracy: 0.9560 - 15s/epoch - 232ms/step
Epoch 4/15
16/16 [==============================] - 1s 33ms/step
63/63 - 14s - loss: 0.1341 - accuracy: 0.9605 - val_loss: 0.2086 - val_accuracy: 0.9480 - 14s/epoch - 225ms/step
Epoch 5/15
16/16 [==============================] - 1s 32ms/step
63/63 - 14s - loss: 0.1291 - accuracy: 0.9598 - val_loss: 0.2122 - val_accuracy: 0.9540 - 14s/epoch - 222ms/step
16/16 [==============================] - 1s 31ms/step - loss: 0.2019 - accuracy: 0.9560
---Starting trial: run

C:\Users\chiez\anaconda3\envs\fashion_labeling\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/15
16/16 [==============================] - 1s 30ms/step
63/63 - 15s - loss: 0.4422 - accuracy: 0.8186 - val_loss: 0.2233 - val_accuracy: 0.9240 - 15s/epoch - 245ms/step
Epoch 2/15
16/16 [==============================] - 1s 36ms/step
63/63 - 16s - loss: 0.1716 - accuracy: 0.9443 - val_loss: 0.2269 - val_accuracy: 0.9560 - 16s/epoch - 250ms/step
Epoch 3/15
16/16 [==============================] - 1s 34ms/step
63/63 - 16s - loss: 0.1514 - accuracy: 0.9513 - val_loss: 0.2155 - val_accuracy: 0.9600 - 16s/epoch - 248ms/step
Epoch 4/15
16/16 [==============================] - 1s 33ms/step
63/63 - 15s - loss: 0.1455 - accuracy: 0.9573 - val_loss: 0.2110 - val_accuracy: 0.9580 - 15s/epoch - 233ms/step
Epoch 5/15
16/16 [==============================] - 1s 30ms/step
63/63 - 13s - loss: 0.1449 - accuracy: 0.9548 - val_loss: 0.2021 - val_accuracy: 0.9520 - 13s/epoch - 214ms/step
Epoch 6/15
16/16 [==============================] - 1s 31ms/step
63/63 - 14s - loss: 0.1386 - accuracy: 0.9585

C:\Users\chiez\anaconda3\envs\fashion_labeling\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model_1/hparam_tuning'


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [37]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model_1/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
